In [1]:
import pandas as pd
import pickle
from tqdm import tqdm

In [2]:
matches = pickle.load(open('../data/processed-data/data-extraction-2.pkl', 'rb'))

In [3]:
delivery_df = pd.DataFrame()

for index in tqdm(range(len(matches))):
    row = matches.iloc[index]
    ball_of_match = []
    over = []
    ball_in_over = []
    batsman = []
    bowler = []
    runs = []
    wickets_left = []
    teams = []
    batting_team = []
    match_id = []
    city = []
    venue = []
    innings = row['innings'][0]['1st innings']
    last_runs: int = 0
    ball_count: int = 0
    wickets_count:int = 0

    # was here, please continue from here
    for ball in innings['deliveries']:
        for ball_number, ball_details in ball.items():
            last_runs += ball_details['runs']['total']
            if 'extras' in ball_details and list(ball_details['extras'].keys())[0] in ['wides', 'noballs']:
                continue
            match_id.append(row['match_id'])
            batting_team.append(innings['team'])
            teams.append(row['info.teams'])
            ball_count += 1
            over.append(ball_count // 6)
            ball_in_over.append(ball_count % 6)
            ball_of_match.append(ball_number)
            batsman.append(ball_details['batsman'])
            bowler.append(ball_details['bowler'])
            runs.append(last_runs)
            last_runs = 0
            city.append(row['info.city'])
            venue.append(row['info.venue'])
            if 'wicket' in ball_details:
                wickets_count += 1
            wickets_left.append(10 - wickets_count)
        
    loop_df = pd.DataFrame({
        'match_id': match_id,
        'city': city,
        'venue': venue,
        'teams': teams,
        'batting_team': batting_team,
        'ball_of_match': ball_of_match,
        'over': over,
        'ball': ball_in_over,
        'batsman': batsman,
        'bowler': bowler,
        'runs': runs,
        'wickets_left': wickets_left
    })

    delivery_df = pd.concat([delivery_df, loop_df], ignore_index=True)

  0%|          | 0/959 [00:00<?, ?it/s]

100%|██████████| 959/959 [00:06<00:00, 151.77it/s]


In [4]:
def get_bowling_team(row):
    for team in row['teams']:
        if team != row['batting_team']:
            return team

delivery_df['bowling_team'] = delivery_df.apply(get_bowling_team, axis=1)

In [5]:
delivery_df.drop(columns=['teams'], inplace=True)

In [6]:
top_teams = [
    'India',
    'Australia',
    'England',
    'South Africa',
    'Pakistan',
    'West Indies',
    'Sri Lanka',
    'New Zealand',
    'Bangladesh',
    'Afghanistan'
]

delivery_df = delivery_df[delivery_df['batting_team'].isin(top_teams)]
delivery_df = delivery_df[delivery_df['bowling_team'].isin(top_teams)]

In [7]:
delivery_df

,match_id,city,venue,batting_team,ball_of_match,over,ball,batsman,bowler,runs,wickets_left,bowling_team
0,1,Wellington,Westpac Stadium,New Zealand,0.1,0,1,MJ Guptill,DJ Willey,2,10,England
1,1,Wellington,Westpac Stadium,New Zealand,0.2,0,2,MJ Guptill,DJ Willey,0,10,England
2,1,Wellington,Westpac Stadium,New Zealand,0.3,0,3,MJ Guptill,DJ Willey,1,10,England
3,1,Wellington,Westpac Stadium,New Zealand,0.4,0,4,C Munro,DJ Willey,0,10,England
4,1,Wellington,Westpac Stadium,New Zealand,0.5,0,5,C Munro,DJ Willey,0,10,England
...,...,...,...,...,...,...,...,...,...,...,...,...
112061,1428,London,Kennington Oval,South Africa,19.2,19,2,MV Boucher,FH Edwards,6,3,West Indies
112062,1428,London,Kennington Oval,South Africa,19.3,19,3,MV Boucher,FH Edwards,1,3,West Indies
112063,1428,London,Kennington Oval,South Africa,19.4,19,4,J Botha,FH Edwards,3,3,West Indies
112064,1428,London,Kennington Oval,South Africa,19.5,19,5,MV Boucher,FH Edwards,0,3,West Indies


In [8]:
pickle.dump(delivery_df, open('../data/processed-data/data-extraction-3.pkl', 'wb'))